In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib

import geopandas
import pandas as pd
import numpy as np
import folium
import gc
from folium.plugins import TimeSliderChoropleth
from scipy.signal import find_peaks

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_bacterias = pd.read_csv('datos/dataset_procesado_bacterias_resistentes.csv')

In [3]:
df_bacterias.head()

,Unnamed: 0,Sample_ID,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
0,0,ERR1549286,2015.0,UK,Europe,0,>256,0,0.016,0,...,-5.965784,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,ERR1549290,2015.0,UK,Europe,0,>256,0,0.004,0,...,-7.965784,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,ERR1549291,2015.0,UK,Europe,0,>256,0,0.006,0,...,-7.380822,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,ERR1549287,2015.0,UK,Europe,0,>256,0,0.006,0,...,-7.380822,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,ERR1549288,2015.0,UK,Europe,0,>256,0,0.008,0,...,-6.965784,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_paises_iso = pd.read_csv('datos/paises.csv')

In [5]:
df_paises_iso.head()

,nombre,name,nom,iso2,iso3,phone_code
0,Afganistán,Afghanistan,Afghanistan,AF,AFG,93
1,Albania,Albania,Albanie,AL,ALB,355
2,Alemania,Germany,Allemagne,DE,DEU,49
3,Andorra,Andorra,Andorra,AD,AND,376
4,Angola,Angola,Angola,AO,AGO,244


In [6]:
df_bacterias.columns.values

array(['Unnamed: 0', 'Sample_ID', 'Year', 'Country', 'Continent',
       'Beta.lactamase', 'Azithromycin', 'Ciprofloxacin', 'Ceftriaxone',
       'Cefixime', 'Tetracycline', 'Penicillin', 'NG_MAST', 'Group',
       'azm_mic', 'cip_mic', 'cro_mic', 'cfx_mic', 'tet_mic', 'pen_mic',
       'log2_azm_mic', 'log2_cip_mic', 'log2_cro_mic', 'log2_cfx_mic',
       'log2_tet_mic', 'log2_pen_mic', 'azm_sr', 'cip_sr', 'cro_sr',
       'cfx_sr', 'tet_sr', 'pen_sr'], dtype=object)

In [7]:
# Creamos los sets
d_name = dict()

for i in range(len(df_paises_iso)):
    d_name[df_paises_iso.iloc[i][4]] = df_paises_iso.iloc[i][1]

d_name

{'AFG': 'Afghanistan',
 'ALB': 'Albania',
 'DEU': 'Germany',
 'AND': 'Andorra',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ATA': 'Antarctica',
 'ATG': 'Antigua and Barbuda',
 'SAU': 'Saudi Arabia',
 'DZA': 'Algeria',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ABW': 'Aruba',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AZE': 'Azerbaijan',
 'BEL': 'Belgium',
 'BHS': 'Bahamas',
 'BHR': 'Bahrain',
 'BGD': 'Bangladesh',
 'BRB': 'Barbados',
 'BLZ': 'Belize',
 'BEN': 'Benin',
 'BTN': 'Bhutan',
 'BLR': 'Belarus',
 'MMR': 'Myanmar',
 'BOL': 'Bolivia',
 'BIH': 'Bosnia and Herzegovina',
 'BWA': 'Botswana',
 'BRA': 'Brazil',
 'BRN': 'Brunei',
 'BGR': 'Bulgaria',
 'BFA': 'Burkina Faso',
 'BDI': 'Burundi',
 'CPV': 'Cape Verde',
 'KHM': 'Cambodia',
 'CMR': 'Cameroon',
 'CAN': 'Canada',
 'TCD': 'Chad',
 'CHL': 'Chile',
 'CHN': 'China',
 'CYP': 'Cyprus',
 'VAT': 'Vatican City State',
 'COL': 'Colombia',
 'COM': 'Comoros',
 'COG': 'Republic of the Congo',
 'COD': 'Democratic Republic of the Congo',
 'PRK': 'No

In [8]:
# Creamos los sets
d_country_id = dict()

for i in range(len(df_paises_iso)):
    d_country_id[df_paises_iso.iloc[i][1]] = i

d_country_id

{'Afghanistan': 0,
 'Albania': 1,
 'Germany': 2,
 'Andorra': 3,
 'Angola': 4,
 'Anguilla': 5,
 'Antarctica': 6,
 'Antigua and Barbuda': 7,
 'Saudi Arabia': 8,
 'Algeria': 9,
 'Argentina': 10,
 'Armenia': 11,
 'Aruba': 12,
 'Australia': 13,
 'Austria': 14,
 'Azerbaijan': 15,
 'Belgium': 16,
 'Bahamas': 17,
 'Bahrain': 18,
 'Bangladesh': 19,
 'Barbados': 20,
 'Belize': 21,
 'Benin': 22,
 'Bhutan': 23,
 'Belarus': 24,
 'Myanmar': 25,
 'Bolivia': 26,
 'Bosnia and Herzegovina': 27,
 'Botswana': 28,
 'Brazil': 29,
 'Brunei': 30,
 'Bulgaria': 31,
 'Burkina Faso': 32,
 'Burundi': 33,
 'Cape Verde': 34,
 'Cambodia': 35,
 'Cameroon': 36,
 'Canada': 37,
 'Chad': 38,
 'Chile': 39,
 'China': 40,
 'Cyprus': 41,
 'Vatican City State': 42,
 'Colombia': 43,
 'Comoros': 44,
 'Republic of the Congo': 45,
 'Democratic Republic of the Congo': 46,
 'North Korea': 47,
 'South Korea': 48,
 'Ivory Coast': 49,
 'Costa Rica': 50,
 'Croatia': 51,
 'Cuba': 52,
 'Curaçao': 53,
 'Denmark': 54,
 'Dominica': 55,
 'Ecu

In [9]:
def get_hex_colors(df, data_to_color, cmap = matplotlib.cm.Reds, log = False):
    
    '''
    This function takes the following arguments
        1. df:pandas DataFrame with the data.
        2. data_to_color: the column name with data based on which we want to create the color scale.
        3. cmap: colors you want to plot. You can use this to communicate different messages. For example: greens --> good, greys --> deaths.
                default is matplotlib.cm.Reds
                more about colormaps: https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
        3. log: if data has huge outliers, we can create the color map with a logarithic normalization. This way, the outliers won't "pale" our other data.
                default is False.
        
    '''
    
    cmap = cmap # define the color pallete you want. You can use Reds, Blues, Greens etc
    my_values = df[data_to_color] # get the value you wan to convert to colors
    
    mini = min(my_values) # get the min to normalize
    maxi= max(my_values) # get the max to normalize
    
    LOGMIN = 0.01 # arbitrary lower bound for log scale
    
    if log: 
        norm = matplotlib.colors.LogNorm(vmin=max(mini,LOGMIN), vmax=maxi) # normalize log data
    else:
        norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi) # create a color range
        
    colors = {value:matplotlib.colors.rgb2hex(cmap(norm(value))[:3]) for value in sorted(list(set(my_values)))} # create a dictionary with the total_infected or deaths as keys and colors as values
    
    return colors

In [10]:
def get_hex_colors_2(value, cats):
    '''
    Color paletter used from this website:
    
    https://colorbrewer2.org/#type=sequential&scheme=Reds&n=9
    
    The color selection will be based on the percentile each value is in.
    '''
    if value == 0:
        return "#FFFFFF"
    elif value in cats[0]:
        return "#fff5f0"
    elif value in cats[1]:
        return "#fee0d2"
    elif value in cats[2]:
        return "#fcbba1"
    elif value in cats[3]:
        return "#fc9272"
    elif value in cats[4]:
        return "#fb6a4a"
    elif value in cats[5]:
        return "#ef3b2c"
    elif value in cats[6]:
        return "#cb181d"
    elif value in cats[7]:
        return "#a50f15"
    elif value in cats[8]:
        return "#67000d"
    else:
        return "#000000"

In [11]:
df_bact_conutry = df_bacterias.groupby(['Country', 'Year'],
                                       as_index=False).count()[[
    'Country', 'Year', 'Sample_ID']].rename(columns={
    'Sample_ID': 'Casos'        
})

In [12]:
df_bact_conutry = df_bact_conutry[df_bact_conutry['Year'] > 2006]

In [17]:
def reemplazar_nomb_paises(pais):
    if pais == 'Brasil':
        return 'Brazil'
    elif pais == 'GuineaBissau':
        return 'Guinea-Bissau'
    elif pais == 'HongKong':
        return 'Hong Kong'
    elif pais == 'Ivory_Coast':
        return 'Ivory Coast'
    elif pais == 'The Netherlands':
        return 'Netherlands'
    elif pais == 'UK':
        return 'United Kingdom'
    elif pais == 'USA':
        return 'United States of America'
    else:
        return pais

In [18]:
# tranformaciones de variables
df_bact_conutry['Country'] = df_bact_conutry['Country'].apply(reemplazar_nomb_paises)

In [19]:
df_bact_conutry.head()

,Country,Year,Casos
2,Argentina,2008.0,1
3,Armenia,2008.0,1
9,Austria,2010.0,2
10,Austria,2013.0,54
11,Belarus,2009.0,10


In [20]:
df_bact_conutry.fillna(0, inplace=True)
df_bact_conutry.shape

(126, 3)

In [21]:
df_bact_conutry["Country_for_Folium"] = df_bact_conutry["Country"].map(d_country_id)

In [22]:
df_bact_conutry.head()

,Country,Year,Casos,Country_for_Folium
2,Argentina,2008.0,1,10.0
3,Armenia,2008.0,1,11.0
9,Austria,2010.0,2,14.0
10,Austria,2013.0,54,14.0
11,Belarus,2009.0,10,24.0


In [23]:
df_bact_conutry.dropna(axis=0, inplace=True)
df_bact_conutry.head()

,Country,Year,Casos,Country_for_Folium
2,Argentina,2008.0,1,10.0
3,Armenia,2008.0,1,11.0
9,Austria,2010.0,2,14.0
10,Austria,2013.0,54,14.0
11,Belarus,2009.0,10,24.0


In [24]:
df_bact_conutry['Country_for_Folium'] = df_bact_conutry['Country_for_Folium'].astype('int')

In [25]:
df_bact_conutry.head()

,Country,Year,Casos,Country_for_Folium
2,Argentina,2008.0,1,10
3,Armenia,2008.0,1,11
9,Austria,2010.0,2,14
10,Austria,2013.0,54,14
11,Belarus,2009.0,10,24


In [26]:
df_bact_conutry.shape

(125, 4)

In [27]:
def correct_date(year):    
    year = str(round(year))
    month = '01'
    day = '01'        
    return "/".join([day, month, year])

In [28]:
df_bact_conutry['NEW_DATE'] = df_bact_conutry['Year'].apply(correct_date)

In [29]:
df_bact_conutry.head()

,Country,Year,Casos,Country_for_Folium,NEW_DATE
2,Argentina,2008.0,1,10,01/01/2008
3,Armenia,2008.0,1,11,01/01/2008
9,Austria,2010.0,2,14,01/01/2010
10,Austria,2013.0,54,14,01/01/2013
11,Belarus,2009.0,10,24,01/01/2009


In [30]:
df_bact_conutry.drop(['Year'], axis=1, inplace=True)
df_bact_conutry["DATE"] = pd.to_datetime(df_bact_conutry["NEW_DATE"], format='%d/%m/%Y')

In [31]:
df_bact_conutry.head()

,Country,Casos,Country_for_Folium,NEW_DATE,DATE
2,Argentina,1,10,01/01/2008,2008-01-01
3,Armenia,1,11,01/01/2008,2008-01-01
9,Austria,2,14,01/01/2010,2010-01-01
10,Austria,54,14,01/01/2013,2013-01-01
11,Belarus,10,24,01/01/2009,2009-01-01


In [32]:
df_bact_conutry["DATE_for_Folium"] = (df_bact_conutry["DATE"].astype('int64')// 10**9).astype('U10')
df_bact_conutry.head()

,Country,Casos,Country_for_Folium,NEW_DATE,DATE,DATE_for_Folium
2,Argentina,1,10,01/01/2008,2008-01-01,1199145600
3,Armenia,1,11,01/01/2008,2008-01-01,1199145600
9,Austria,2,14,01/01/2010,2010-01-01,1262304000
10,Austria,54,14,01/01/2013,2013-01-01,1356998400
11,Belarus,10,24,01/01/2009,2009-01-01,1230768000


In [33]:
df_bact_conutry.rename(columns={'Country_for_Folium':'id'}, inplace=True)
# df_bact_conutry["id"].astype(np.int16)

In [34]:
df_bact_conutry = df_bact_conutry[['id', 'Country', 'DATE_for_Folium', 'Casos']]
df_bact_conutry.head()

,id,Country,DATE_for_Folium,Casos
2,10,Argentina,1199145600,1
3,11,Armenia,1199145600,1
9,14,Austria,1262304000,2
10,14,Austria,1356998400,54
11,24,Belarus,1230768000,10


In [35]:
df_bact_conutry.rename(columns={'DATE_for_Folium': 'DATE'}, inplace=True)
df_bact_conutry.head()

,id,Country,DATE,Casos
2,10,Argentina,1199145600,1
3,11,Armenia,1199145600,1
9,14,Austria,1262304000,2
10,14,Austria,1356998400,54
11,24,Belarus,1230768000,10


In [36]:
# rellenamos la serie
countries = list(np.unique(df_bact_conutry.Country.values))
dates = list(np.unique(df_bact_conutry.DATE.values))
for country in countries:
    for date in dates:
        if (len(df_bact_conutry[(df_bact_conutry['Country'] == country) & df_bact_conutry['DATE'] == date]) == 0):
            df_bact_conutry = pd.concat([df_bact_conutry,
                                         pd.DataFrame({'id': [d_country_id[country]],
                                                       'Country': [country],
                                                       'DATE': [date],
                                                       'Casos': [0]})],
                                         ignore_index=True)

In [37]:
df_bact_conutry.shape

(631, 4)

In [38]:
gdf = geopandas.read_file('datos/ne_110m_admin_0_countries.json')

In [39]:
gdf['id'] = gdf['NAME'].map(d_country_id)

In [40]:
gdf.dropna(axis=0, inplace=True)

In [41]:
gdf = gdf[['id', 'geometry']]

In [42]:
gdf.head()

,id,geometry
0,69.0,"POLYGON ((180.00000 -16.06712, 180.00000 -16.5..."
1,224.0,"POLYGON ((33.90369 -0.95000, 34.07260 -1.05982..."
3,37.0,"POLYGON ((-122.83997 49.00000, -122.97418 49.0..."
4,64.0,"POLYGON ((-122.83997 49.00000, -120.00000 49.0..."
5,126.0,"POLYGON ((87.35992 49.21497, 86.59875 48.54916..."


In [43]:
gdf["geometry"] = gdf["geometry"].simplify(0.1, preserve_topology = False)

In [44]:
gdf['id'] = gdf["id"].astype(int)
gdf.head()

,id,geometry
0,69,"POLYGON ((180.00000 -16.06712, 180.00000 -16.5..."
1,224,"POLYGON ((33.90369 -0.95000, 37.69867 -3.09699..."
3,37,"POLYGON ((-122.83997 49.00000, -125.62457 50.4..."
4,64,"POLYGON ((-122.83997 49.00000, -95.15906 49.00..."
5,126,"POLYGON ((87.35992 49.21497, 86.59875 48.54916..."


In [45]:
m = folium.Map(location = (40, 0), zoom_start = 5.5)

folium.Choropleth(
    geo_data = gdf,
    name = 'choropleth',
    data = df_bact_conutry[df_bact_conutry["DATE"] == max(df_bact_conutry["DATE"])],
    columns = ['id', 'Casos'],
    key_on='feature.properties.id',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name = 'Mapa segun especimenes de bacteria resistente'
).add_to(m)

m

In [46]:
#----------------------------------------------------------------------------------
data_to_color = "Casos"
cats, bins =  pd.qcut(df_bact_conutry[data_to_color].unique()[np.argsort(df_bact_conutry[data_to_color].unique())], q = 9, retbins = True)
cats = cats.unique()

#----------------------------------------------------------------------------------

In [47]:
bins

array([  0.        ,   4.55555556,   9.11111111,  14.33333333,
        27.33333333,  40.55555556,  49.33333333,  59.66666667,
       111.55555556, 176.        ])

In [48]:
# value we will iterate in order to create the styledict
# Ordenamos
df_bact_conutry.sort_values(['id'], inplace=True)
countries = list(df_bact_conutry["id"].unique())
dates = list(df_bact_conutry["DATE"].unique())

In [49]:
# create the color dict and color column
df_bact_conutry["COLORS"] = df_bact_conutry[data_to_color].apply(get_hex_colors_2, args = [cats]) # we create a colum in the df so that we can iterate and create the styledict

# creates the styledict for the map
styledict = {}

# iterate the populate the styledict
for country in countries:
    try:
        styledict[str(country)] = {date: {'color': df_bact_conutry[(df_bact_conutry["id"] == country)
                                                                   & (df_bact_conutry["DATE"] == date)]["COLORS"].values[0],
                                          'opacity': 0.6} for date in dates}
    except:
        styledict[str(country)] = {date: {'color': '#fff5f0', 'opacity': 0.6}}
    
# creates and renders the Folium map
m = folium.Map(location=(40, 0), tiles='OpenStreetMap', zoom_start=2)

g = TimeSliderChoropleth(
    gdf.set_index("id").to_json(), # get's the coordinates for each id 
    styledict = styledict # styledict contains for each id the timestamp and the color to plot.
)

m.add_child(g)


from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>  
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <style>body: {overflow-y: hidden;}</style>
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#FFFFFF;opacity:0.6;'></span>No hay casos</li>
    <li><span style='background:#fff5f0;opacity:0.6;'></span>Por debajo de 4</li>
    <li><span style='background:#fee0d2;opacity:0.6;'></span>Entre 4 y 7</li>
    <li><span style='background:#fcbba1;opacity:0.6;'></span>Entre 7 y 11</li>
    <li><span style='background:#fc9272;opacity:0.6;'></span>Entre 11 20</li>
    <li><span style='background:#fb6a4a;opacity:0.6;'></span>Entre 20 y 37</li>
    <li><span style='background:#ef3b2c;opacity:0.6;'></span>Entre 37 y 44</li>
    <li><span style='background:#cb181d;opacity:0.6;'></span>Entre 44 y 50</li>
    <li><span style='background:#a50f15;opacity:0.6;'></span>Entre 50 y 57</li>
    <li><span style='background:#67000d;opacity:0.6;'></span>Más de 57</li>
    <li><span style='background:#000000;opacity:0.6;'></span>Otros</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)
m

In [51]:
m.save('html/mapa_mundial_bacterias_resistentes.html')